In [14]:
import os
from dotenv import load_dotenv
import pandas as pd
pd.set_option('display.max_colwidth', 0)
import requests
import praw
from datetime import datetime, timedelta

#Import credentials from .env file
load_dotenv()
redid = os.getenv("redid")
redtok = os.getenv("redtok")

# Create a Reddit instance. Replace 'my_client_id', 'my_client_secret', 'my_user_agent' with your own information.
reddit = praw.Reddit(client_id=redid, 
                     client_secret=redtok, 
                     user_agent='Last Year\'s News, by Bearly Thinking')

start_date = int((datetime.now() - timedelta(days=365)).timestamp())
end_date = int((datetime.now() - timedelta(days=364)).timestamp())

subreddit = reddit.subreddit("news")

# Scraping the top posts of all time
posts = subreddit.top(time_filter = "year", limit = None)
 
news = {
    "Title": [], "Score": [], "Upvote Ratio": [],"Created On":[], "Post URL": []
    }

for post in posts:
    # Date of each posts' creation
    date = post.created_utc
    if date > start_date:
        # Title of each post
        news["Title"].append(post.title)
        # The score of a post
        news["Score"].append(post.score)
        # Upvote Ratio of a post
        news["Upvote Ratio"].append(post.upvote_ratio)
        # Date the post was Created
        news["Created On"].append(post.created_utc)
        # URL of each post
        news["Post URL"].append(post.url)

# Saving the data in a pandas dataframe
news_posts = pd.DataFrame(news)
news_posts['Subreddit'] = "News"

# Repeat for WorldNews subreddit
subreddit = reddit.subreddit("worldnews")

# Scraping the top posts of all time
posts = subreddit.top(time_filter = "year", limit = None)
 
worldnews = {
    "Title": [], "Score": [], "Upvote Ratio": [],"Created On":[], "Post URL": []
    }

for post in posts:
    # Date of each posts' creation
    date = post.created_utc
    if date > start_date:
        # Title of each post
        worldnews["Title"].append(post.title)
        # The score of a post
        worldnews["Score"].append(post.score)
        # Upvote Ratio of a post
        worldnews["Upvote Ratio"].append(post.upvote_ratio)
        # Date the post was Created
        worldnews["Created On"].append(post.created_utc)
        # URL of each post
        worldnews["Post URL"].append(post.url)

# Saving the data in a pandas dataframe
worldnews_posts = pd.DataFrame(worldnews)
worldnews_posts['Subreddit'] = "World News"

# Concatenate the two dataframes
all_posts = pd.concat([news_posts, worldnews_posts], axis=0)
all_posts['Created On'] = pd.to_datetime(all_posts['Created On'],  unit='s')

lastyearsnews  = all_posts[
    (all_posts['Created On'] >= "{}".format(pd.to_datetime(start_date,  unit='s'))) 
    & 
    (all_posts['Created On'] < "{}".format(pd.to_datetime(end_date,  unit='s')))
    ].sort_values(by='Score', ascending = False)
lastyearsnews

,Title,Score,Upvote Ratio,Created On,Post URL,Subreddit
267,Secret Service cannot recover texts; no new details for Jan. 6 committee,48406,0.90,2022-07-19 17:14:37,https://www.washingtonpost.com/nation/2022/07/19/secret-service-texts/,News
329,"17 members of Congress arrested during Supreme Court protest, Capitol police say - CBS News",43824,0.88,2022-07-19 20:22:11,https://www.cbsnews.com/news/representatives-congress-arrested-today-supreme-court-abortion-alexandria-ocasio-cortez-carolyn-maloney-2022-07-19/,News
864,"Heat wave kills more than 1,700 people in Spain and Portugal",18918,0.96,2022-07-20 10:45:39,https://www.axios.com/2022/07/18/heat-wave-europe-death-toll,World News
966,"NATO leader tells Europe to ""stop complaining"" and help Ukraine",16166,0.91,2022-07-19 19:43:23,https://www.newsweek.com/nato-leader-tells-europe-stop-complaining-help-ukraine-1726105,World News
